In [1]:
import TableParser
import pandas as pd
import tempfile, zipfile
import os
import numpy as np

In [498]:
class TestTableParser(TableParser.TableParserItem):
    def __init__(self):
        pass
    
    def parse(self, series: pd.Series):
        mistake = {
            'Jan':'01','Feb': '02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sept':'09','Oct':'10','Nov':'11','Dec':'12',
            'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','G': '6','F': '7','Z': '2','Q': '2','B': '8','O': '0','D': '0','E': '3','A': '4','S': '5',
            r'[ -/]|[:-@]|[\[-_]|[{-~]': '-',
        }
        for key, val in mistake.items():
            series = series.str.strip().str.replace(key, val, regex=True)
        return (pd.to_datetime(series, errors="ignore"), [])

    def fix_xlsx(self, in_file):
        tmpfd, tmp = tempfile.mkstemp(dir=os.path.dirname(in_file))
        os.close(tmpfd)
        filename = '[Content_Types].xml'
        data = ''
        with zipfile.ZipFile(in_file, 'r') as zin:
            with zipfile.ZipFile(tmp, 'w') as zout:
                for item in zin.infolist():
                    if item.filename != filename:
                        zout.writestr(item, zin.read(item.filename))
                    else:
                        data = zin.read(filename).decode()
        os.remove(in_file)
        os.rename(tmp, in_file)
        data = data.replace('/xl/sharedStrings.xml', '/xl/SharedStrings.xml')
        with zipfile.ZipFile(in_file, mode='a', compression=zipfile.ZIP_DEFLATED) as zf:
            zf.writestr(filename, data)

    ###############################################
    # to find the x,y coordinate of row with date #
    
    def find_coordinate(self, x, y,df): # and defind axis
        test_list =[[0,0]]
        count_X_axis,count_Y_axis,main_axis = 0,0,"Y axis"
        
        for i in range(df.shape[0]):
            if len(test_list) == 11: 
                break
            for j in range(df.shape[1]):
                if  type(str(df.iloc[i,j])) == type("string") and len(str(df.iloc[i,j])) > 5:
                    if type(testTableParser.parse(pd.Series([str(df.iloc[i,j])]))[0][0]) == type(pd.to_datetime('31.01.2002')):
                        test_list.append([i,j])
                        if test_list[-1][1] > test_list[-2][1]:
                            count_X_axis += 1
                        if test_list[-1][0] > test_list[-2][0]:
                            count_Y_axis += 1  
                        if len(test_list) == 11:
                            break
#         print(test_list)
        if count_X_axis > count_Y_axis:
            main_axis = "X axis"
            
        axis = test_list[1]
        if main_axis == "X axis":
            for coordinate in test_list:
                if coordinate[0] != axis[0]:
                    axis = coordinate
        elif main_axis == "Y axis":
            for coordinate in test_list:
                if coordinate[1] != axis[1]:
                    axis = coordinate
                
        return axis[0],axis[1],main_axis
    
   
    
    def y_axis_solution(self,df,data_row_num,data_col_num,true_place):
        table = df.iloc[data_row_num:, data_col_num:].reset_index(drop=True)  # <--- all useful data
        date = df.iloc[data_row_num:,data_col_num].astype(str)
        date = testTableParser.parse(date)
        table.iloc[0:,0] = date[0].reset_index(drop=True)
        index_of_table = list(table.columns)
        num = index_of_table[0]
        index_of_table.pop(0)
        index_of_table.insert(true_place,num)
        table = table[index_of_table]
        min_height = 0
        if data_row_num > 3:min_height = data_row_num- 3
        elif data_row_num< 3 and data_row_num >1: min_height = data_row_num -1
        list_index =[]

        for i in range(data_col_num,df.shape[1]):
            buffer = ""
            if pd.isnull(df.iloc[min_height,i]) and len(list_index)!= 0:
        #         print(df.iloc[min_height,i])
                buffer = "upWord_"
            for j in range(min_height,data_row_num):
                buffer += str(df.iloc[j,i]).replace('\n',' ')+ "__"
                
            if buffer.strip().replace("nan__","") == "upWord_":
                list_index.append("-----")
            else: list_index.append(buffer.replace("nan__","").strip())
        #             list_index.append(buffer.replace("nan__","")[:-2])
        #     print(list_index)
            if list_index[-1][:7] == "upWord_":
                if list_index[-2] == "-----":
                    index = list_index[-3].find("_")
                    list_index[-1] = list_index[-3][:index+2]+list_index[-1][7:]
                else: 
                    index = list_index[-2].find("_")
                    list_index[-1] = list_index[-2][:index+2]+list_index[-1][7:]
        list_index = pd.Series(list_index).str[:-2]
        index_of_table.remove(0)
        index_of_table.insert(true_place,0)
        list_index = list_index.reindex(index_of_table)
        table.columns = list_index 
#         list_index.head(15)
        return table
    
    def x_axis_solution(self,df,data_row_num,data_col_num,true_place):
#         print(data_row_num,data_col_num)

        table = df.iloc[data_row_num:, data_col_num:].reset_index(drop=True)  # <--- all useful data
        date = df.iloc[data_row_num, data_col_num:].astype(str)  # <--- Series with date (01.03.2016;...)
        date = testTableParser.parse(date)
        table.iloc[0] = date[0]
        index_of_table = list(table.index)
        index_of_table.remove(0)
        index_of_table.insert(true_place,0)
        table = table.reindex(index_of_table)
#         print(list(table.index))
        min_deep = 0
        if data_col_num > 3:min_deep = data_col_num - 3
        elif data_col_num< 3 and data_col_num >1: min_deep = data_col_num -1
        list_index =[]
        for i in range(data_row_num,df.shape[0]):
            buffer = ""
            if pd.isnull(df.iloc[i,min_deep]) and len(list_index)!= 0:
                buffer = "upWord_"
            for j in range(min_deep,data_col_num):
                buffer +=str(df.iloc[i,j]).replace('\n',' ')+"__"
                
            if buffer.strip().replace("nan__","") == "upWord_":
                list_index.append("-----")
            else: list_index.append(buffer.replace("nan__","").strip())
#             print(list_index)
            if list_index[-1][:7] == "upWord_":
                index = list_index[-2].find("_")
                list_index[-1] = list_index[-2][:index+2]+list_index[-1][7:]
        list_index = pd.Series(list_index).str[:-2]
        
        list_index = list_index.reindex(index_of_table)
        table = table.set_index(list_index)
        return table
    
    def all_action(self,df): # <-----    drive method
        data_row_num, data_col_num,main_axis = self.find_coordinate(0,0,df) # <-- find coordinate of dates
#         print(data_row_num, data_col_num,main_axis)
        if main_axis == "X axis":
            index = list(df.index)
#             print(index)
#             print(data_row_num)
            index.remove(data_row_num)
            index.insert(0,data_row_num)
            df = df.reindex(index)
            true_place = data_row_num
            data_row_num, data_col_num,main_axis = self.find_coordinate(0,0,df) # <-- find coordinate of dates
            
        elif main_axis == "Y axis":
            index = list(df.columns)
            index.remove(data_col_num)
            index.insert(0,data_col_num)
            df = df[index]
            true_place = data_col_num
            data_row_num, data_col_num,main_axis = self.find_coordinate(0,0,df) # <-- find coordinate of dates
            
        if main_axis == "X axis":
            return self.x_axis_solution(df,data_row_num,data_col_num,true_place)
        elif main_axis == "Y axis":
            return self.y_axis_solution(df,data_row_num,data_col_num,true_place)
        return df.sort_index(ascending=True)

In [499]:
testTableParser = TestTableParser()
# testTableParser.fix_xlsx('file1.xlsx')
df = pd.read_excel("file22_2.xlsx",header=None)
testTableParser.all_action(df)

,---,---,---,Показатели__ЕдИзм,Дата на начало периода,Наличие кур-несушек на начало периода__гол.,Пало__норм__гол.,Пало__факт__гол.,Пало__норм__%,Пало__факт__%,...,Среднемесячное поголовье__гол.,Яиц на начальную курицу-несушку__норм__шт.,Яиц на начальную курицу-несушку__факт__шт.,Яиц на среднюю курицу-несушку__норм__шт.,Яиц на среднюю курицу-несушку__факт__шт.,Яйценоскость__норм__%,Яйценоскость__факт__%,Средний вес 1-ой головы__кг,Однородность %,Средний вес 1-го яйца__г
0,NaN,NaN,NaN,NaN,2016-01-03,24490,98,10,0.40,0.04,...,875,20.20,0.68,21.00,18.93,75.00,67.62,2.018,75.00,65.6
1,NaN,NaN,NaN,NaN,2016-02-03,24486,98,4,0.40,0.02,...,875,20.20,0.69,21.00,19.35,75.00,69.10,2.018,75.00,65.6
2,NaN,NaN,NaN,NaN,2016-03-03,24474,98,12,0.40,0.05,...,874,20.20,0.68,21.00,18.95,75.00,67.66,2.018,75.00,65.6
3,NaN,NaN,NaN,NaN,2016-04-03,24470,122,4,0.50,0.02,...,874,18.10,0.67,19.00,18.74,67.86,66.94,2.018,75.00,65.6
4,NaN,NaN,NaN,NaN,2016-05-03,24469,122,1,0.50,NaN,...,874,18.10,0.67,19.00,18.74,67.86,66.94,2.018,75.00,65.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,NaN,NaN,NaN,NaN,2018-10-12,29999,150,10,0.50,0.03,...,1071,18.10,0.73,19.00,16.80,67.86,60.00,1.930,80.00,NaN
920,NaN,NaN,NaN,NaN,2018-11-12,29995,150,4,0.50,0.01,...,1071,18.10,0.71,19.00,16.13,67.86,57.61,1.930,80.00,NaN
921,NaN,NaN,NaN,NaN,2018-12-12,18092,90,5,0.50,0.03,...,646,18.10,0.65,19.00,24.51,67.86,87.55,1.930,80.00,NaN
922,NaN,NaN,NaN,NaN,2018-12-13,7119,107,7,1.50,0.10,...,254,17.00,0.29,18.00,27.61,64.29,98.61,1.930,80.00,NaN


In [452]:
df

,0,1,2,3,4,5,6,7,8,9,...,173,174,175,176,177,178,179,180,181,182
0,NaN,Стадо РМ №1,NaN,NaN,1 партия,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Воз-ст 1 партии,Воз-ст 2 партии,Показатель,1 корпус (кур),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,№ корпуса,"Поголовье, гол",NaN,"Выбраковка, гол",NaN,"Ср вес 1 гол, гр",NaN,Расход корма,...,Выбраковка,NaN,Ср вес 1 гол,NaN,Расход корма,Продуктивность,Вес яйца,Вода,"t, С утро",Влажность
3,NaN,NaN,Дата,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,2019-12-28 00:00:00,8471,NaN,NaN,NaN,41.1,NaN,338.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,144,144,2020-05-20 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,145,145,2020-05-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,146,146,2020-05-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,147,147,2020-05-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [233]:
table

,4,5,6,7,8,9,10,11,12,13,...,361,362,363,364,365,366,367,368,369,370
1,0,1,2,3,4,5,6,7,8,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-12-28 00:00:00,2019-12-29 00:00:00,2019-12-30 00:00:00,2019-12-31 00:00:00,2020-01-01 00:00:00,2020-01-02 00:00:00,2020-01-03 00:00:00,2020-01-04 00:00:00,2020-01-05 00:00:00,2020-01-06 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8471,8467,8461,8451,8446,8444,8439,8438,8438,8438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print("as_asd".find("_"))

In [ ]:
df = pd.read_excel("file22.xlsx")
xls = pd.ExcelFile('file22.xlsx')
xls.sheet_names

In [ ]:
sheet_to_df_map = {}
for sheet_name in xls.sheet_names:
    sheet_to_df_map[sheet_name] = xls.parse(sheet_name)

In [ ]:
list(sheet_to_df_map.values())[0]

In [ ]:
type(df.iloc[3,2])

In [ ]:
type(pd.to_datetime('01.01.2021'))